<a href="https://colab.research.google.com/github/matchbou/kaggle-study-20190410/blob/master/Getting%20started%20with%20Time-series%20features%20_%20Kaggle.pdf-201904131613.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/danofer/getting-started-with-time-series-features

# Getting started with Time-series features _ Kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"matchbou","key":"b332de8173f61d7f1a987ce8d0178075"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!pip install kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
# kaggleデータ置き場とjupyernoteのカレントディレクトリフォルダを作成

!mkdir ./input
!mkdir ./notebook

In [0]:
#osライブラリ読込してinputに移動⇒kaggleデータもダウンロード　⇒ notebookフォルダに移動
import os
os.chdir('input')



In [8]:
#https://www.kaggle.com/c/titanic/data
!kaggle competitions download -c demand-forecasting-kernels-only

  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 60.3MB/s]
  0% 0.00/135k [00:00<?, ?B/s]
100% 135k/135k [00:00<00:00, 39.0MB/s]
  0% 0.00/3.08M [00:00<?, ?B/s]
100% 3.08M/3.08M [00:00<00:00, 101MB/s]


In [9]:
!pwd
!ls -a

/content/input
.  ..  sample_submission.csv.zip  test.csv.zip	train.csv.zip


In [10]:
!unzip sample_submission.csv.zip -d ./
!unzip test.csv.zip -d ./
!unzip train.csv.zip -d ./

Archive:  sample_submission.csv.zip
  inflating: ./sample_submission.csv  
Archive:  test.csv.zip
  inflating: ./test.csv              
Archive:  train.csv.zip
  inflating: ./train.csv             


In [0]:
os.chdir('../notebook')

In [12]:
#上記操作の結果以下のディレクトリ配置となる

# /content/notebook このipynbのカレントディレクトリ
# /content/input カグルからダウンロードしたcsvファイルの置き場所

!pwd
!ls -a ../input


/content/notebook
.   sample_submission.csv      test.csv      train.csv
..  sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [0]:
#ここからカーネルの内容に踏み込む

In [14]:
import pandas as pd
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sample = pd.read_csv('../input/sample_submission.csv')

['train.csv', 'sample_submission.csv', 'test.csv.zip', 'test.csv', 'train.csv.zip', 'sample_submission.csv.zip']


In [15]:
#merge train and test for easier feature engineering.

train.columns

Index(['date', 'store', 'item', 'sales'], dtype='object')

In [16]:
test.columns

Index(['id', 'date', 'store', 'item'], dtype='object')

In [17]:
print("train shape:", train.shape)
print("Test shape:", test.shape)
df = pd.concat([train,test])
print(df.shape)
df.head()

train shape: (913000, 4)
Test shape: (45000, 4)
(958000, 5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,date,id,item,sales,store
0,2013-01-01,NaN,1,13.0,1
1,2013-01-02,NaN,1,11.0,1
2,2013-01-03,NaN,1,14.0,1
3,2013-01-04,NaN,1,13.0,1
4,2013-01-05,NaN,1,10.0,1


In [0]:
#naive datetime features:

df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)


df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.dayofweek
df['year'] = df['date'].dt.year
# df['date'].dt.
df['week_of_year']  = df.date.dt.weekofyear


In [0]:
df.set_index("date",inplace=True)


In [0]:
#Add historical / seasonal features


df["median-store_item-month"] = df.groupby(['month',"item","store"])["sales"].transform("median")
df["mean-store_item-week"] = df.groupby(['week_of_year',"item","store"])["sales"].transform("mean")
df["item-month-sum"] = df.groupby(['month',"item"])["sales"].transform("sum") # total sales of that item  for all stores
df["store-month-sum"] = df.groupby(['month',"store"])["sales"].transform("sum") # total sales of that store  for all items

In [0]:
# get shifted features for grouped data. Note need to sort first! 
df['store_item_shifted-90'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(90)) # sales for that item 90 days = 3 months ago
df['store_item_shifted-180'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(180)) # sales for that item 180 days = 3 months ago
df['store_item_shifted-365'] = df.groupby(["item","store"])['sales'].transform(lambda x:x.shift(365)) # sales for that 1 year  ago

df["item-week_shifted-90"] = df.groupby(['week_of_year',"item"])["sales"].transform(lambda x:x.shift(12).sum()) # shifted total sales for that item 12 weeks (3 months) ago
df["store-week_shifted-90"] = df.groupby(['week_of_year',"store"])["sales"].transform(lambda x:x.shift(12).sum()) # shifted total sales for that store 12 weeks (3 months) ago
df["item-week_shifted-90"] = df.groupby(['week_of_year',"item"])["sales"].transform(lambda x:x.shift(12).mean()) # shifted mean sales for that item 12 weeks (3 months) ago
df["store-week_shifted-90"] = df.groupby(['week_of_year',"store"])["sales"].transform(lambda x:x.shift(12).mean()) # shifted mean sales for that store 12 weeks (3 months) ago

In [22]:
df.tail()

,id,item,sales,store,month,weekday,year,week_of_year,median-store_item-month,mean-store_item-week,item-month-sum,store-month-sum,store_item_shifted-90,store_item_shifted-180,store_item_shifted-365,item-week_shifted-90,store-week_shifted-90
date,,,,,,,,,,,,,,,,,
2018-03-27,44995.0,50,NaN,10,3,1,2018,13,67.0,68.771429,92609.0,411975.0,63.0,82.0,66.0,61.417143,54.788571
2018-03-28,44996.0,50,NaN,10,3,2,2018,13,67.0,68.771429,92609.0,411975.0,59.0,90.0,60.0,61.417143,54.788571
2018-03-29,44997.0,50,NaN,10,3,3,2018,13,67.0,68.771429,92609.0,411975.0,74.0,103.0,73.0,61.417143,54.788571
2018-03-30,44998.0,50,NaN,10,3,4,2018,13,67.0,68.771429,92609.0,411975.0,62.0,99.0,68.0,61.417143,54.788571
2018-03-31,44999.0,50,NaN,10,3,5,2018,13,67.0,68.771429,92609.0,411975.0,82.0,71.0,69.0,61.417143,54.788571


In [0]:
#We should do one hot encoding at this point on the store and ite mIDs to avoid silly range based features.

#split our data for modelling

col = [i for i in df.columns if i not in ['date','id']]
y = 'sales'

In [24]:
train.columns

Index(['date', 'store', 'item', 'sales'], dtype='object')

In [25]:
train = df.loc[~df.sales.isna()]
print("new train",train.shape)
test = df.loc[df.sales.isna()]
print("new test",test.shape)

new train (913000, 17)
new test (45000, 17)


In [0]:
#Evaluation should use temporal train test split or temporal CV

train_x, train_cv, y, y_cv = train_test_split(train[col],train[y], test_size=0.15, random_state=42)
# train_x, train_cv, y, y_cv = TimeSeriesSplit(train[col],train[y], test_size=0.1, random_state=42)



In [0]:
def XGB_regressor(train_X, train_y, test_X, test_y, feature_names=None, seed_val=2017, num_rounds=500):
    param = {}
    param['objective'] = 'reg:linear'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = 'mae'
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model    

In [28]:
model = XGB_regressor(train_X = train_x, train_y = y, test_X = train_cv, test_y = y_cv)
y_test = model.predict(xgb.DMatrix(test[col]), ntree_limit = model.best_ntree_limit)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mae:46.5709	test-mae:46.6042
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 20 rounds.
[1]	train-mae:41.9145	test-mae:41.9444
[2]	train-mae:37.7236	test-mae:37.7475
[3]	train-mae:33.9518	test-mae:33.973
[4]	train-mae:30.5571	test-mae:30.5759
[5]	train-mae:27.5019	test-mae:27.5188
[6]	train-mae:24.7527	test-mae:24.7659
[7]	train-mae:22.278	test-mae:22.2899
[8]	train-mae:20.0505	test-mae:20.0606
[9]	train-mae:18.0459	test-mae:18.0546
[10]	train-mae:16.2416	test-mae:16.2493
[11]	train-mae:14.6187	test-mae:14.6242
[12]	train-mae:13.1572	test-mae:13.162
[13]	train-mae:11.8419	test-mae:11.8458
[14]	train-mae:10.6581	test-mae:10.6613
[15]	train-mae:9.5928	test-mae:9.59569
[16]	train-mae:8.63406	test-mae:8.63654
[17]	train-mae:7.77105	test-mae:7.77293
[18]	train-mae:6.99453	test-mae:6.99617
[19]	train-mae:6.29563	test-mae:6.29698
[20]	train-mae:5.6668	test-mae:5.66798
[21]	train-mae:5.10097	test-mae:5.

In [0]:
sample['sales'] = y_test
sample.to_csv('simple_starter.csv', index=False)

In [0]:
from google.colab import files
files.download("simple_starter.csv") # 作成したファイルをダウンロード